
# パディングとストライド

:label: `sec_padding`

 :numref: `fig_correlation`の畳み込みの例を思い出してください。入力の高さと幅は両方とも 3 で、畳み込みカーネルの高さと幅は両方とも 2 で、次元 $2\times2$ の出力表現が生成されます。入力形状が $n_h\times n_w$ で、畳み込みカーネル形状が $k_h\times k_w$ であると仮定すると、出力形状は $(n_h-k_h+1) \times (n_w-k_w+1)$ になります。畳み込みカーネルをシフトできるのは、畳み込みを適用するピクセルがなくなるまでのみです。

以下では、パディングやストライド畳み込みなど、出力のサイズをより詳細に制御できるさまざまな手法を検討します。動機として、カーネルの幅と高さは通常 $1$ より大きいため、多くの連続した畳み込みを適用した後、入力よりもかなり小さい出力が得られる傾向があることに注意してください。 $240 \times 240$ ピクセルの画像から始めると、$5 \times 5$ 畳み込みの $10$ レイヤーによって画像が $200 \times 200$ ピクセルに縮小され、画像の $30 %$ が切り取られ、画像上の興味深い情報はすべて消去されます。元の画像の境界。*パディングは*、この問題を処理するための最も一般的なツールです。また、元の入力解像度が手に負えないと判断した場合など、次元を大幅に削減する必要がある場合もあります。*ストライド畳み込みは*、このような場合に役立つ一般的なテクニックです。


In [1]:
import torch
from torch import nn


## パディング

上で説明したように、畳み込みレイヤーを適用する際の注意が必要な問題の 1 つは、画像の周囲のピクセルが失われる傾向があることです。ピクセル使用率をコンボリューション カーネル サイズと画像内の位置の関数として表す :numref: `img_conv_reuse`を考えてみましょう。隅のピクセルはほとんど使用されていません。 

![](../img/conv-reuse.svg) :label: `img_conv_reuse`

通常、小さなカーネルを使用するため、特定の畳み込みでは数ピクセルしか失われませんが、多くの連続した畳み込みレイヤーを適用すると、損失が増加する可能性があります。この問題に対する簡単な解決策の 1 つは、入力画像の境界の周囲にフィラーのピクセルを追加して、画像の有効サイズを増やすことです。通常、追加のピクセルの値はゼロに設定されます。 :numref: `img_conv_pad`では、 $3 \times 3$ 入力をパディングし、そのサイズを $5 \times 5$ に増やします。対応する出力は $4 \times 4$ 行列に増加します。影付きの部分は、最初の出力要素と、出力計算に使用される入力およびカーネル テンソル要素です: $0\times0+0\times1+0\times2+0\times3=0$。 

![](../img/conv-pad.svg):ラベル: `img_conv_pad`

一般に、合計 $p_h$ 行のパディング (およそ上半分と下半分) と合計 $p_w$ 列のパディング (およそ左側と右側の半分) を追加すると、出力は形状は

$$(n_h-k_h+p_h+1)\times(n_w-k_w+p_w+1).$$

これは、出力の高さと幅がそれぞれ $p_h$ と $p_w$ だけ増加することを意味します。

多くの場合、入力と出力に同じ高さと幅を与えるために $p_h=k_h-1$ と $p_w=k_w-1$ を設定する必要があります。これにより、ネットワーク構築時に各層の出力形状を予測しやすくなります。ここで $k_h$ が奇数であると仮定して、高さの両側に $p_h/2$ 行を埋め込みます。 $k_h$ が偶数の場合、入力の上部に $\lceil p_h/2\rceil$ 行を埋め込み、入力の下部に $\lfloor p_h/2\rfloor$ 行を埋め込むことが考えられます。幅の両側も同様に埋めていきます。

 CNN は通常、1、3、5、7 などの奇数の高さと幅の値を持つ畳み込みカーネルを使用します。奇数のカーネル サイズを選択すると、上下に同じ数の行をパディングしながら次元を維持できるという利点があります。左右の列数は同じです。

さらに、奇数のカーネルとパディングを使用して次元を正確に保持するこの方法は、事務的な利点をもたらします。任意の 2 次元テンソル`X`について、カーネルのサイズが奇数で、すべての辺のパディング行と列の数が同じで、入力と同じ高さと幅の出力が生成される場合、出力`Y[i, j]` `X[i, j]`を中心とするウィンドウを使用した入力とコンボリューション カーネルの相互相関によって計算されます。

次の例では、高さと幅が 3 の 2 次元畳み込み層を作成し、(**すべての側面に 1 ピクセルのパディングを適用します**)、高さと幅が 8 の入力を与えると、高さと幅が次のようになります。出力も8です。


In [2]:
# We define a helper function to calculate convolutions. It initializes the
# convolutional layer weights and performs corresponding dimensionality
# elevations and reductions on the input and output
def comp_conv2d(conv2d, X):
    # (1, 1) indicates that batch size and the number of channels are both 1
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    # Strip the first two dimensions: examples and channels
    return Y.reshape(Y.shape[2:])

# 1 row and column is padded on either side, so a total of 2 rows or columns
# are added
conv2d = nn.LazyConv2d(1, kernel_size=3, padding=1)
X = torch.rand(size=(8, 8))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])


コンボリューションカーネルの高さと幅が異なる場合、**高さと幅に異なるパディング番号を設定することで、出力と入力の高さと幅を同じにすることができます。** 】


In [3]:
# We use a convolution kernel with height 5 and width 3. The padding on either
# side of the height and width are 2 and 1, respectively
conv2d = nn.LazyConv2d(1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])


## ストライド

相互相関を計算するときは、入力テンソルの左上隅にある畳み込みウィンドウから開始して、それをすべての位置にわたって下と右の両方にスライドさせます。前の例では、デフォルトで一度に 1 つの要素をスライドさせました。ただし、計算効率のため、またはダウンサンプリングしたいため、ウィンドウを一度に複数の要素を移動し、中間の位置をスキップすることがあります。これは、基礎となる画像の広い領域をキャプチャするため、コンボリューション カーネルが大きい場合に特に便利です。

スライドごとに走査される行と列の数を*stride*と呼びます。これまでは、高さと幅の両方に 1 のストライドを使用してきました。場合によっては、より大きな歩幅を使用したい場合もあります。 :numref: `img_conv_stride`垂直方向に 3、水平方向に 2 のストライドを持つ 2 次元の相互相関演算を示します。影付きの部分は出力要素と、出力の計算に使用される入力およびカーネル テンソル要素です: $0\times0+0\times1+1\times2+2\times3=8$, $0\times0+6\times1+0 \times2+0\times3=6$。最初の列の 2 番目の要素が生成されると、畳み込みウィンドウが 3 行下にスライドすることがわかります。最初の行の 2 番目の要素が生成されると、畳み込みウィンドウは 2 列右にスライドします。畳み込みウィンドウが入力上で 2 列右にスライドし続けると、(パディングの別の列を追加しない限り) 入力要素がウィンドウを満たすことができないため、出力はありません。 

![](../img/conv-stride.svg) :label: `img_conv_stride`

一般に、高さのストライドが $s_h$、幅のストライドが $s_w$ の場合、出力形状は次のようになります。

 $$\lfloor(n_h-k_h+p_h+s_h)/s_h\rfloor \times \lfloor(n_w-k_w+p_w+s_w)/s_w\rfloor.$$

 $p_h=k_h-1$ および $p_w=k_w-1$ と設定すると、出力形状は $\lfloor(n_h+s_h-1)/s_h\rfloor \times \lfloor(n_w+s_w- 1)/s_w\rフロア$。さらに一歩進んで、入力の高さと幅が高さと幅のストライドで割り切れる場合、出力の形状は $(n_h/s_h) \times (n_w/s_w)$ になります。

以下では、 [**高さと幅の両方のストライドを 2 に設定**] し、入力の高さと幅を半分にします。


In [4]:
conv2d = nn.LazyConv2d(1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])


(**もう少し複雑な例**) を見てみましょう。


In [5]:
conv2d = nn.LazyConv2d(1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape

torch.Size([2, 2])


## 要約と考察

パディングにより、出力の高さと幅が増加する可能性があります。これは、出力の望ましくない縮小を避けるために、出力に入力と同じ高さと幅を与えるためによく使用されます。さらに、すべてのピクセルが同じ頻度で使用されることが保証されます。通常、入力の高さと幅の両側で対称的なパディングを選択します。この場合、$(p_h, p_w)$ パディングを参照します。最も一般的には $p_h = p_w$ を設定しますが、この場合は単純にパディング $p$ を選択すると述べます。

同様の規則がストライドにも適用されます。水平方向のストライド $s_h$ と垂直方向のストライド $s_w$ が一致する場合、単にストライド $s$ について話します。ストライドにより出力の解像度が低下する可能性があります。たとえば、$n &gt; 1$ の場合、出力の高さと幅が入力の高さと幅の $1/n$ にのみ縮小されます。デフォルトでは、パディングは 0、ストライドは 1 です。

これまで説明してきたパディングはすべて、単に画像をゼロで拡張するものでした。これは簡単に達成できるため、計算上大きな利点があります。さらに、追加のメモリを割り当てることなく、このパディングを暗黙的に利用するように演算子を設計できます。同時に、CNN は「空白」がどこにあるかを学習するだけで、画像内の暗黙の位置情報をエンコードできるようになります。ゼロパディングの代替手段は数多くあります。 :citet: `Alsallakh.Kokhlikyan.Miglani.ea.2020` 、代替手段の広範な概要が提供されています (アーティファクトが発生しない限り、ゼロ以外のパディングを使用する明確なケースはありませんが)。

## 演習
1. このセクションの最後のコード例でカーネル サイズ $(3, 5)$、パディング $(0, 1)$、ストライド $(3, 4)$ を指定すると、出力形状を計算して出力形状が一致しているかどうかを確認します。実験結果。
1. オーディオ信号の場合、ストライド 2 は何に対応しますか?
1. ミラー パディング、つまり、テンソルを拡張するために境界値が単純にミラーリングされるパディングを実装します。
1. ストライドが 1 より大きい場合、計算上どのような利点がありますか?
1. ストライドが 1 より大きい場合、統計的にどのような利点があるでしょうか?
1.  $\frac{1}{2}$ のストライドをどのように実装しますか?それは何に相当しますか?これはいつ役立つでしょうか?



[ディスカッション](https://discuss.d2l.ai/t/68)
